In [1]:
!python --version


Python 3.13.5


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki.name

'wikipedia'

In [5]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000014FF36F1BD0>, search_kwargs={})

In [11]:
from langchain_core.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [12]:
retriever_tool.name

'langsmith_search'

In [14]:
tools = [wiki, arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Administrator\\Desktop\\rag1\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000014FF6A37560>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<la

In [16]:
from dotenv import load_dotenv
import os

load_dotenv()  # Loads variables from .env file
groq_api_key = os.environ.get("GROQ_API_KEY")

In [52]:
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=0, model_name="llama-3.3-70b-versatile")

In [53]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([("system","You are a helpful Search Assistant"),
                                           ("human","{input}"),
                                           ("placeholder","{agent_scratchpad}")])

In [63]:
from langchain.agents import create_agent
search_agent = create_agent(
    llm,
    tools,
    system_prompt="You are a helpful assistant"
)


In [65]:
search_agent.invoke(
    {"messages": [{"role": "user", "content": "Tell me about Langsmith from langsmith search"}]}
)

{'messages': [HumanMessage(content='Tell me about Langsmith from langsmith search', additional_kwargs={}, response_metadata={}, id='9aab0342-4b4b-4f9c-8420-935401ce45aa'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'mp1z7emdx', 'function': {'arguments': '{"query":"Langsmith information"}', 'name': 'langsmith_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 534, 'total_tokens': 552, 'completion_time': 0.048602527, 'prompt_time': 0.045895726, 'queue_time': 0.049398524, 'total_time': 0.094498253}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--10919899-ee1d-4de6-9f3c-50cc3422a8a1-0', tool_calls=[{'name': 'langsmith_search', 'args': {'query': 'Langsmith information'}, 'id': 'mp1z7emdx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 534, 'output_tokens'

In [71]:
search_agent.invoke(
    {"messages": [{"role": "user", "content": "Tell about langsmith"}]}
)

{'messages': [HumanMessage(content='Tell about langsmith', additional_kwargs={}, response_metadata={}, id='d23c3430-d127-4190-95c0-a399efed8883'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'y6rpm29jf', 'function': {'arguments': '{"query":"LangSmith information"}', 'name': 'langsmith_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 529, 'total_tokens': 547, 'completion_time': 0.050103977, 'prompt_time': 0.045016544, 'queue_time': 0.045558086, 'total_time': 0.095120521}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--8ef31e1f-8ad3-4271-959e-b6f3c5d65c46-0', tool_calls=[{'name': 'langsmith_search', 'args': {'query': 'LangSmith information'}, 'id': 'y6rpm29jf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 529, 'output_tokens': 18, 'total_tokens': 547